# Data Loading, Cleaning, and Representation

In [152]:
# Import necessary libraries
import pandas as pd
import re
import numpy as np

In [154]:
# Function for load the dataset
def LoadDataset(file_name):
    try:
        data = pd.read_csv(file_name)
        print(f'Loaded the data {file_name} successfully.')
        return data
    except Exception as e:
        print(f'Error: {e}')
        return None

In [9]:
# Call the function
data = LoadDataset("job_descriptions.csv")

Loaded the data job_descriptions.csv successfully.


In [156]:
# Function for clean the dataset
def DataCleaning(data):
    # Remove the unnecessary columns
    columns_to_drop = ['Job Id','Experience','Qualifications','Country','latitude','longitude','Work Type'
                  ,'Company Size','Preference','Contact Person','Contact','Role','Job Portal','Job Description',
                  'Benefits','skills','Responsibilities','Company Profile']
    data = data.drop(columns = columns_to_drop)
    
    # Rename the columns
    data = data.rename(columns = {'Job Title':'Title','location':'Location','Salary Range':'Average Salary','Job Posting Date':'Date Posted'})
    
    # Columns in new order
    new_order = ['Title', 'Company','Location','Average Salary','Date Posted']
    data = data[new_order]
    
    # Check for data types
    print(f'**Data types before treatment:** \n\n{data.dtypes}\n')
    
    # Treatment for wrong data types
    data['Date Posted'] = pd.to_datetime(data['Date Posted'],errors = 'coerce')
    data['Average Salary'] = data['Average Salary'].apply(
    lambda x: (sum([int(i) for i in re.findall(r'\d+', x)]) / len(re.findall(r'\d+', x)) * 1000)
    if pd.notna(x) and re.search(r'\d+', x) else np.nan
    )
    data['Average Salary'] = pd.to_numeric(data['Average Salary'],errors = 'coerce')
    
    # Data types after treatment
    print(f'**Data types after treatment:** \n\n{data.dtypes}\n')
    
    # Check for duplicate values
    print(f'**Duplicate Values:** \n{data.duplicated().sum()}\n')
    
    # Check for missing values
    print(f'**Missing Values:** \n\n{data.isna().sum()}\n')
    
    # Outlier Detection
    Q1 = data['Average Salary'].quantile(0.25)
    Q2 = data['Average Salary'].quantile(0.5)
    Q3 = data['Average Salary'].quantile(0.75)
    IQR = Q3 - Q1
    LW = Q1 - (1.5 * IQR)
    UW = Q3 + (1.5 * IQR)
    Outliers = ((data['Average Salary'] < LW) | (data['Average Salary'] > UW))
    if Outliers.sum() > 0:
        print(f'Average Salary has {Outliers.sum()} Outliers.')
    else:
        print(f'**Average Salary has 0 Outliers.**')
    return data

In [118]:
# Call the function
cleaned_data = DataCleaning(data)

**Data types before treatment:** 

Title             object
Company           object
Location          object
Average Salary    object
Date Posted       object
dtype: object

**Data types after treatment:** 

Title                     object
Company                   object
Location                  object
Average Salary           float64
Date Posted       datetime64[ns]
dtype: object

**Duplicate Values:** 
0

**Missing Values:** 

Title             0
Company           0
Location          0
Average Salary    0
Date Posted       0
dtype: int64

**Average Salary has 0 Outliers.**


In [160]:
# Export the clean data into csv format
cleaned_data.to_csv('clean_job_descriptions.csv',index=False)

In [162]:
# Function for clean data Overview 
def CleanDataRepresentation(clean_data_file):
    try:
        dataframe = pd.read_csv(clean_data_file, parse_dates=['Date Posted'])
        print(f'Clean file {clean_data_file} successfully uploaded.\n')
        print(f'**Display sample rows for understanding:**\n')
        display(dataframe.head())
        print(f'**Statistical summary of data:**\n')
        display(dataframe.describe(include = 'all'))
        print(f'**Summary of data:**\n')
        return dataframe.info()
    except Exception as e:
        print(f'Error: {e}')
        

In [164]:
# Call the function
CleanDataRepresentation('clean_job_descriptions.csv')

Clean file clean_job_descriptions.csv successfully uploaded.

**Display sample rows for understanding:**



,Title,Company,Location,Average Salary,Date Posted
0,Digital Marketing Specialist,Icahn Enterprises,Douglas,79000.0,2022-04-24
1,Web Developer,PNC Financial Services Group,Ashgabat,86000.0,2022-12-19
2,Operations Manager,United Services Automobile Assn.,Macao,82500.0,2022-09-14
3,Network Engineer,Hess,Porto-Novo,78000.0,2023-02-25
4,Event Manager,Cairn Energy,Santiago,75500.0,2022-10-11


**Statistical summary of data:**



,Title,Company,Location,Average Salary,Date Posted
count,1615940,1615940,1615940,1.615940e+06,1615940
unique,147,888,214,NaN,NaN
top,UX/UI Designer,DTE Energy,Seoul,NaN,NaN
freq,48551,1957,15104,NaN,NaN
mean,NaN,NaN,NaN,8.249115e+04,2022-09-15 02:07:38.874463744
min,NaN,NaN,NaN,6.750000e+04,2021-09-15 00:00:00
25%,NaN,NaN,NaN,7.600000e+04,2022-03-16 00:00:00
50%,NaN,NaN,NaN,8.250000e+04,2022-09-15 00:00:00
75%,NaN,NaN,NaN,8.900000e+04,2023-03-17 00:00:00
max,NaN,NaN,NaN,9.750000e+04,2023-09-15 00:00:00


**Summary of data:**

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1615940 entries, 0 to 1615939
Data columns (total 5 columns):
 #   Column          Non-Null Count    Dtype         
---  ------          --------------    -----         
 0   Title           1615940 non-null  object        
 1   Company         1615940 non-null  object        
 2   Location        1615940 non-null  object        
 3   Average Salary  1615940 non-null  float64       
 4   Date Posted     1615940 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), object(3)
memory usage: 61.6+ MB
